# Company Problem

In this notebook, our goal is to familiarize you with Logica while also enabling you to contrast Logica with SQL.

### Package Installation

First-time users need to install the following packages

In [ ]:
# !python3 -m pip install logica
# !python3 -m pip install pandas
# !pip install ipython-sql

After running the cell above, please restart the kernel to move on

### Package Import

You can disregard the message when executing the cell below

In [ ]:
from logica import colab_logica
import pandas as pd
import sqlite3

In [ ]:
colab_logica.SetPreamble(
    '@Engine("sqlite"); @AttachDatabase("mydata", "my_database.db");'
)

In [ ]:
%load_ext sql

### Database Creation

Logica is typically run on a database. Since we only possess CSV files, we need first create a database.

In [ ]:
Dep = pd.read_csv("datasets/Department.csv")
Dpd = pd.read_csv("datasets/Dependent.csv")
Dep_loc = pd.read_csv("datasets/Dept_Location.csv")
Emp = pd.read_csv("datasets/Employee.csv", dtype={"Super_ssn": "str"})
Proj = pd.read_csv("datasets/Project.csv")
Works_On = pd.read_csv("datasets/Works_On.csv", dtype={"Pno": "str"})

In [ ]:
Dep.columns = Dep.columns.str.lower()
Dpd.columns = Dpd.columns.str.lower()
Dep_loc.columns = Dep_loc.columns.str.lower()
Emp.columns = Emp.columns.str.lower()
Proj.columns = Proj.columns.str.lower()
Works_On.columns = Works_On.columns.str.lower()

In [ ]:
conn = sqlite3.connect("my_database.db")

Dep.to_sql("Department", conn, if_exists="replace", index=False)
Dpd.to_sql("Dependent", conn, if_exists="replace", index=False)
Dep_loc.to_sql("Dept_Location", conn, if_exists="replace", index=False)
Emp.to_sql("Employee", conn, if_exists="replace", index=False)
Proj.to_sql("Project", conn, if_exists="replace", index=False)
Works_On.to_sql("Works_On", conn, if_exists="replace", index=False)

conn.close()

### Queries

We've provided a range of queries, in SQL and Logica formats, sourced from the textbook "FUNDAMENTALS OF Database Systems". Consider these as your starting point. Enjoy!

#### Query 0: Retrieve the birth date and address of the employee(s) whose name is 'John B. Smith'.

$\mathrm{\sigma_{Fname='John' \land Minit='B' \land Lname='Smith'}(EMPLOYEE)}$

In [ ]:
%%sql sqlite:///my_database.db
SELECT Bdate, Address
FROM EMPLOYEE
WHERE Fname='John' AND Minit='B' AND Lname='Smith'

In [ ]:
%%logica JohnSmith

JohnSmith(bdate:, address:) :- 
    Employee(bdate:, address:, fname:"John", minit:"B", lname:"Smith");

#### Query 1: Retrieve the name and address of all employees who work for the 'Research' department. 

$\mathrm{\Pi_{Fname, Lname, Address}(\sigma_{Dname='Research'}(DEPARTMENT\bowtie_{Dnumber=Dno}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db
SELECT Fname,Lname,Address
FROM EMPLOYEE,DEPARTMENT
WHERE Dname='Research' AND Dnumber=Dno;

In [ ]:
%%logica ResearchDep

ResearchDep(fname:, lname:, address:):- 
    Employee(fname:, lname:, address:, dno:), 
    Department(dname:"Research", dnumber:dno);

#### Query 2: For every project located in 'Stafford', list the project number, the controlling department number, and the department manager's last name, address, and birth date. 

$\mathrm{\Pi_{Pnumber,Dnum,Lname,Address,Bdate}(((\sigma_{Plocation='Stafford'}(PROJECT))
\bowtie_{Dnum=Dnumber}(DEPARTMENT))\bowtie_{Mgr\_ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db
Select Pnumber,Dnum,Lname,Address,Bdate
From Project,Department,Employee
Where Dnum=Dnumber And Mgr_ssn=Ssn And Plocation='Stafford'

In [ ]:
%%logica StaffordProj

StaffordProj(pnumber:, dnum:, lname:, address:, bdate:):-
    Project(pnumber:, dnum:, plocation:"Stafford"), 
    Department(dnumber:dnum, mgr_ssn:),
    Employee(lname:, address:, ssn:mgr_ssn, bdate:);

#### Query 3: Who is the boss? The manager of the department that an employee works for!

$\mathrm{\Pi_{SSN, Lname, Address}(EMPLOYEE\bowtie_{Super\_Ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT E.SSN as 'Employee', 
       M.SSN as 'Boss'
FROM   Employee E, 
       Employee M
WHERE E.Super_SSN = M.SSN
ORDER BY M.SSN;

In [ ]:
%%logica EmpBoss

EmpBoss(employee:ssn,boss:super_ssn):- 
    Employee(ssn:, super_ssn:),Employee(ssn:super_ssn);

#### Query 4: Make a list of all project numbers for projects that involve an employee whose last name is ‘Smith’, either as a worker or as a manager of the department that controls the project.

$\mathrm{SMITHS(Essn) \leftarrow \pi_{Ssn}(\sigma_{Lname='Smith'}(EMPLOYEE))}$

$\mathrm{SMITH\_WORKER\_PROJS \leftarrow \pi_{Pno}(WORKS\_ON) \ast SMITHS}$

$\mathrm{MGRS \leftarrow \pi_{Lname,Dnumber}(EMPLOYEE \bowtie_{Ssn=Mgr\_ssn}DEPARTMENT)}$

$\mathrm{SMITH\_MANAGED\_DEPTS(Dnum) \leftarrow \pi_{Dnumber}(\sigma_{Lname='Smith'}(MGRS))}$

$\mathrm{SMITH\_MANAGED\_PROJS(Pno) \leftarrow \pi_{Pnumber}(SMITH\_MANAGED\_DEPTS * PROJECT)}$

$\mathrm{RESULT \leftarrow (SMITH\_WORKER\_PROJS \cup SMITH\_MANAGED\_PROJS)}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT DISTINCT P.Pnumber
FROM Project AS P
JOIN Department AS D ON P.Dnum = D.Dnumber
JOIN Employee AS E ON D.Mgr_ssn = E.ssn
WHERE E.Lname = 'Smith'

UNION

SELECT DISTINCT P.Pnumber
FROM Project AS P
JOIN Works_on AS W ON P.Pnumber = W.Pno
JOIN Employee AS E ON W.Essn = E.Ssn
WHERE E.Lname = 'Smith'

In [ ]:
%%logica SmithProj

SmithProj(pnumber:pno):-
    Works_On(essn:, pno:),
    Employee(lname:"Smith",ssn:essn);
    
SmithProj(pnumber:):-
    Project(pnumber:, dnum:),
    Department(dnumber:dnum,mgr_ssn:),
    Employee(ssn:mgr_ssn, lname:"Smith");

#### Query 5: For each employee, retrieve the employee’s first and last name and the first and last name of his or her immediate supervisor.

$\mathrm{\Pi_{E\_fname, E\_lname, S\_fname, S\_lname}(EMPLOYEE\bowtie_{Super\_Ssn=Ssn}(EMPLOYEE))}$

In [ ]:
%%sql sqlite:///my_database.db

SELECT E.Fname as E_fname, E.Lname as E_lname, S.Fname as S_fname, S.Lname as S_lname
FROM EMPLOYEE AS E, EMPLOYEE AS S
WHERE E.Super_ssn = S.Ssn;

In [ ]:
%%logica EmpBossName

EmpBossName(e_fname:x, e_lname:y, s_fname:m,s_lname:l):- 
    Employee(fname:x, lname:y,super_ssn:),
    Employee(fname:m, lname:l,Ssn:super_ssn);

#### Query 6: Select all combinations of EMPLOYEE Ssn and DEPARTMENT Dname (Q10) in the database

$\mathrm{\sigma_{Ssn, Dname}EMPLOYEE \times DEPARTMENT}$

In [ ]:
%%sql sqlite:///my_database.db
    
SELECT Ssn, Dname
FROM EMPLOYEE, DEPARTMENT

In [ ]:
%%logica SsnDname

SsnDname(ssn:, dname:):-
    Employee(ssn:), Department(dname:);

#### Query 7: Show the resulting salaries if every employee working on the ‘ProductX’ project is given a 10% raise

$\mathrm{EMP\_PROJX \leftarrow \sigma_{Pname='ProjectX'}(WORKS\_ON \bowtie_{Pno=Pnumber} PROJECT)}$

$\mathrm{RESULT(Fname, Lname, Increased\_sal) \leftarrow \pi_{Fname, Lname, 1.1*Salary}(EMPLOYEE \bowtie_{Ssn=Essn} EMP\_PROJX)}$

In [ ]:
%%sql sqlite:///my_database.db
    
SELECT E.Fname, E.Lname, 1.1 * E.Salary AS Increased_sal
FROM EMPLOYEE AS E, WORKS_ON AS W, PROJECT AS P
WHERE E.Ssn = W.Essn AND W.Pno = P.Pnumber AND
P.Pname = 'ProductX';

In [ ]:
%%logica IncreSal

IncreSal(fname:, lname:, increased_sal:salary*1.1):-
    Employee(fname:, lname:,ssn:, salary:),
    Works_On(essn:ssn, pno:), 
    Project(pname:"ProductX",pnumber:pno);

## Logica Exercises

#### Exercise: Week 9/Q4: Retrieve the name and address of all employees who work for the 'Administration' department. 

In [ ]:
%%logica Q4

# Insert your logica query here
Q4(Fname:fn, Lname:ln, Address:ad):-

#### Exercise: Week 9/Q6: Retrieve the name of each employee who has a dependent with the same sex as the employee.

In [ ]:
%%logica Q6

# Insert your logica query here
Q6(Fname:fn, Lname:ln, Dependent_name:dn):-